# Review lab:Structured data model (Relational database)

This Google colaboratory notebook contains Python codes for performing SQL statements. The sqlite3 DB will be used. It is automatically installed with the Google colaboratory platform. See [https://www.sqlite.org/docs.html](https://www.sqlite.org/docs.html) for more information.

Written by Associate Professor Dr. Krung Sinapiromsaran  
Version 0.01:25 June 2024  

## &#48;. Lab Preparation

1. Check the availability of sqlite3.  
2. Download chinook database from this URL [Chinook.zip](http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip). Need to unzip since it is in the zip format.  
3. Set up the connection object variable with the chinook.db file and set up the cursor to pass the SQL statement.  


In [ ]:
# The output should produce "The sqlite3 module is available." if not you will need to install it.
import sys
if 'sqlite3' in sys.modules:
  print("The sqlite3 module is available.")
else:
  print("The sqlite3 module is not available")

The sqlite3 module is available.


In [ ]:
# Get the chinook database from sqlitetutorial and unzip it
!wget http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
!unzip chinook.zip

--2024-06-19 03:01:50--  http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Resolving www.sqlitetutorial.net (www.sqlitetutorial.net)... 104.21.30.141, 172.67.172.250, 2606:4700:3037::6815:1e8d, ...
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|104.21.30.141|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip [following]
--2024-06-19 03:01:50--  https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Connecting to www.sqlitetutorial.net (www.sqlitetutorial.net)|104.21.30.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305596 (298K) [application/zip]
Saving to: ‘chinook.zip’

chinook.zip         100%[===================>] 298.43K  --.-KB/s    in 0.03s   

2024-06-19 03:01:50 (8.87 MB/s) - ‘chinook.zip’ saved [305596/305596]

Archive:  chinook.zip
  inflating: chinook.db              


In [ ]:
# Import sqlite library
import sqlite3

In [ ]:
# Create the database connection with the current chinook.db
dbconnect = sqlite3.connect('chinook.db')
cursor = dbconnect.cursor()

## &#49;. Extract meta-data from SQLite DB

From the current chinook.db, SQL statments for DML (Data manipulation language) can be used to extract instances from a table. Here is the step-by-step instruction.  
* Connect to the given database via the database connection.  
* Define the cursor to represent the pointer to each individual record.  
* Use cursor.fetchone() or cursor.fetchall() to extract a single next record or all records.  


In [ ]:
def list_all_tables_in(db_file):
  """Lists all tables in SQLite3 databases"""
  with sqlite3.connect(db_file) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table_info[0] for table_info in cursor.fetchall() if 'sqlite' not in table_info[0]]
    return tables

In [ ]:
list_all_tables_in('chinook.db')

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks']

There are 11 tables in the chinook database which are stored in `sqlite_master` table. Note that other tables with the phrase 'sqlite' are internal to SQLite3. They will be removed from the table list.  

## &#50;. SQL:Data definition language

**DDL** (Data Definition Language) is used to define and manipulate the structure of objects within a relational database
Functionality:
* Create database objects (CREATE)
* Modify database objects (ALTER)
* Delete database objects (DROP)


In [ ]:
# Create a new relation
cursor.execute('''
    CREATE TABLE IF NOT EXISTS test (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER,
        salary REAL
    )
''')
list_all_tables_in('chinook.db')

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks',
 'test']

In [ ]:
# Add a new column
cursor.execute('ALTER TABLE test ADD COLUMN email TEXT')
list_all_tables_in('chinook.db')

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks',
 'test']

In [ ]:
# Add a new instance
cursor.execute('''
   INSERT INTO test ('name', 'age', 'salary', 'email') VALUES ("Krung", 20, 50000, "krung.s@chula.ac.th")
   ''')

In [ ]:
# Extract the field names
cursor.execute('SELECT * FROM test')
names = [description[0] for description in cursor.description]
print(names)

['id', 'name', 'age', 'salary', 'email']


In [ ]:
# Delete test relation
try:
  cursor.execute("DROP TABLE IF EXISTS test")
  print("Table 'test' dropped successfully.")
except sqlite3.Error as e:
  print(f"Error dropping table: {e}")

Table 'test' dropped successfully.


In [ ]:
# Disconnect
dbconnect.close()

## &#51;. SQL: DML

**DML** (Data Manipulation Language) is used to extract and update tuples

### &#51;.&#49; Show all tuples in a table

To show records of any relations
* Database must be connected.
* Set cursor to access the database (One time only)
* Perform operation per record

In [ ]:
# Create the database connection with the current chinook.db
dbconnect = sqlite3.connect('chinook.db')
cursor = dbconnect.cursor()

In [ ]:
# Show all records in employees table
cursor.execute("SELECT * FROM employees")
_ = [print(row) for row in cursor.fetchall()]

(1, 'Adams', 'Andrew', 'General Manager', None, '1962-02-18 00:00:00', '2002-08-14 00:00:00', '11120 Jasper Ave NW', 'Edmonton', 'AB', 'Canada', 'T5K 2N1', '+1 (780) 428-9482', '+1 (780) 428-3457', 'andrew@chinookcorp.com')
(2, 'Edwards', 'Nancy', 'Sales Manager', 1, '1958-12-08 00:00:00', '2002-05-01 00:00:00', '825 8 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 2T3', '+1 (403) 262-3443', '+1 (403) 262-3322', 'nancy@chinookcorp.com')
(3, 'Peacock', 'Jane', 'Sales Support Agent', 2, '1973-08-29 00:00:00', '2002-04-01 00:00:00', '1111 6 Ave SW', 'Calgary', 'AB', 'Canada', 'T2P 5M5', '+1 (403) 262-3443', '+1 (403) 262-6712', 'jane@chinookcorp.com')
(4, 'Park', 'Margaret', 'Sales Support Agent', 2, '1947-09-19 00:00:00', '2003-05-03 00:00:00', '683 10 Street SW', 'Calgary', 'AB', 'Canada', 'T2P 5G3', '+1 (403) 263-4423', '+1 (403) 263-4289', 'margaret@chinookcorp.com')
(5, 'Johnson', 'Steve', 'Sales Support Agent', 2, '1965-03-03 00:00:00', '2003-10-17 00:00:00', '7727B 41 Ave', 'Calgary', 'A

In [ ]:
# Extract the field names from employees table
print([description[0] for description in cursor.description])

['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo', 'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email']


In [ ]:
# To extract the type of each field, need to get it from pragma_table_info()
cursor.execute("SELECT name, type FROM pragma_table_info('employees')")
_ = [print(row) for row in cursor.fetchall()]

('EmployeeId', 'INTEGER')
('LastName', 'NVARCHAR(20)')
('FirstName', 'NVARCHAR(20)')
('Title', 'NVARCHAR(30)')
('ReportsTo', 'INTEGER')
('BirthDate', 'DATETIME')
('HireDate', 'DATETIME')
('Address', 'NVARCHAR(70)')
('City', 'NVARCHAR(40)')
('State', 'NVARCHAR(40)')
('Country', 'NVARCHAR(40)')
('PostalCode', 'NVARCHAR(10)')
('Phone', 'NVARCHAR(24)')
('Fax', 'NVARCHAR(24)')
('Email', 'NVARCHAR(60)')


In [ ]:
from prettytable import PrettyTable
cursor.execute("SELECT * FROM employees")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+------------+----------+-----------+---------------------+-----------+---------------------+---------------------+-----------------------------+------------+-------+---------+------------+-------------------+-------------------+--------------------------+
| EmployeeId | LastName | FirstName | Title               | ReportsTo | BirthDate           | HireDate            | Address                     | City       | State | Country | PostalCode | Phone             | Fax               | Email                    |
+------------+----------+-----------+---------------------+-----------+---------------------+---------------------+-----------------------------+------------+-------+---------+------------+-------------------+-------------------+--------------------------+
| 1          | Adams    | Andrew    | General Manager     | None      | 1962-02-18 00:00:00 | 2002-08-14 00:00:00 | 11120 Jasper Ave NW         | Edmonton   | AB    | Canada  | T5K 2N1    | +1 (780) 428-9482 | +1 (780) 428-3457 |

### &#51;.&#50;. Projection

A user can project some attributes via `SELECT`.

In [ ]:
# Show only the firstname and the lastname from employees table
cursor.execute("SELECT firstname, lastname FROM employees")
_ = [print(row) for row in cursor.fetchall()]

('Andrew', 'Adams')
('Nancy', 'Edwards')
('Jane', 'Peacock')
('Margaret', 'Park')
('Steve', 'Johnson')
('Michael', 'Mitchell')
('Robert', 'King')
('Laura', 'Callahan')


In [ ]:
# Alternatively alias can be used to represent employees table
cursor.execute("SELECT E.firstname, E.lastname FROM employees E")
_ = [print(row) for row in cursor.fetchall()]

('Andrew', 'Adams')
('Nancy', 'Edwards')
('Jane', 'Peacock')
('Margaret', 'Park')
('Steve', 'Johnson')
('Michael', 'Mitchell')
('Robert', 'King')
('Laura', 'Callahan')


In [ ]:
# Show only the firstname and the lastname in employees table sorted by lastname
cursor.execute("SELECT firstname, lastname FROM employees ORDER BY lastname ASC")
_ = [print(row) for row in cursor.fetchall()]

('Andrew', 'Adams')
('Laura', 'Callahan')
('Nancy', 'Edwards')
('Steve', 'Johnson')
('Robert', 'King')
('Michael', 'Mitchell')
('Margaret', 'Park')
('Jane', 'Peacock')


In [ ]:
# Show five records of the firstname and the lastname in employees table
cursor.execute("SELECT firstname, lastname FROM employees LIMIT 5")
_ = [print(row) for row in cursor.fetchall()]

('Andrew', 'Adams')
('Nancy', 'Edwards')
('Jane', 'Peacock')
('Margaret', 'Park')
('Steve', 'Johnson')


In [ ]:
# Show only distinct titile in employees table
cursor.execute("SELECT DISTINCT title FROM employees")
_ = [print(row) for row in cursor.fetchall()]

('General Manager',)
('Sales Manager',)
('Sales Support Agent',)
('IT Manager',)
('IT Staff',)


### &#51;.&#51;. Selection

Extract tuples that satisfy some conditions.  

In [ ]:
# Show trackid from invoice_items table having unitprice equal to 1.99
cursor.execute("SELECT trackid FROM invoice_items WHERE unitprice == 1.99")
[row for row in cursor.fetchall()]

[(2820,),
 (2826,),
 (2832,),
 (2838,),
 (2844,),
 (2850,),
 (2856,),
 (2862,),
 (2868,),
 (2874,),
 (2883,),
 (2892,),
 (2901,),
 (2910,),
 (2919,),
 (3169,),
 (3178,),
 (3187,),
 (3196,),
 (3205,),
 (3214,),
 (3223,),
 (3232,),
 (3246,),
 (3247,),
 (3248,),
 (3250,),
 (3252,),
 (3338,),
 (3347,),
 (3428,),
 (2821,),
 (2827,),
 (2833,),
 (2839,),
 (2845,),
 (2851,),
 (2860,),
 (2869,),
 (2878,),
 (2887,),
 (2896,),
 (2905,),
 (2914,),
 (2923,),
 (3173,),
 (3182,),
 (3191,),
 (3200,),
 (3209,),
 (3223,),
 (3224,),
 (3227,),
 (3229,),
 (3231,),
 (3233,),
 (3235,),
 (3237,),
 (3241,),
 (3245,),
 (3249,),
 (3342,),
 (3360,),
 (2822,),
 (2828,),
 (2837,),
 (2846,),
 (2855,),
 (2864,),
 (2873,),
 (2882,),
 (2891,),
 (2900,),
 (2909,),
 (2918,),
 (3168,),
 (3177,),
 (3186,),
 (3200,),
 (3201,),
 (3202,),
 (3204,),
 (3206,),
 (3208,),
 (3210,),
 (3212,),
 (3214,),
 (3218,),
 (3222,),
 (3226,),
 (3230,),
 (3234,),
 (3238,),
 (3244,),
 (3250,),
 (3337,),
 (3346,),
 (3364,),
 (2823,),
 (2832,),


In [ ]:
# Show name from tracks table that plays between 100000 and 300000 milliseconds
cursor.execute("SELECT name FROM tracks WHERE milliseconds BETWEEN 100000 AND 300000 LIMIT 10")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+--------------------------+
| Name                     |
+--------------------------+
| Fast As a Shark          |
| Restless and Wild        |
| Put The Finger On You    |
| Let's Get It Up          |
| Inject The Venom         |
| Snowballed               |
| Evil Walks               |
| C.O.D.                   |
| Breaking The Rules       |
| Night Of The Long Knives |
+--------------------------+


### &#51;.&#52;. Rename

Attributes can be renamed.

In [ ]:
# Rename 'name' as 'TrackName' and show only ten of them
cursor.execute("SELECT name AS TrackName FROM tracks LIMIT 10")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+-----------------------------------------+
| TrackName                               |
+-----------------------------------------+
| For Those About To Rock (We Salute You) |
| Balls to the Wall                       |
| Fast As a Shark                         |
| Restless and Wild                       |
| Princess of the Dawn                    |
| Put The Finger On You                   |
| Let's Get It Up                         |
| Inject The Venom                        |
| Snowballed                              |
| Evil Walks                              |
+-----------------------------------------+


## &#52;. SQL with multiple tables

## &#52;.&#49;. Join

Two tables can be joined by some attributes.

In [ ]:
# Show 10 tuples from albums table
cursor.execute("SELECT * FROM albums LIMIT 10")
for row in cursor.fetchall():
  print(row)

(1, 'For Those About To Rock We Salute You', 1)
(2, 'Balls to the Wall', 2)
(3, 'Restless and Wild', 2)
(4, 'Let There Be Rock', 1)
(5, 'Big Ones', 3)
(6, 'Jagged Little Pill', 4)
(7, 'Facelift', 5)
(8, 'Warner 25 Anos', 6)
(9, 'Plays Metallica By Four Cellos', 7)
(10, 'Audioslave', 8)


In [ ]:
# Show 10 tup[es from artists table
cursor.execute("SELECT * FROM artists LIMIT 10")
for row in cursor.fetchall():
  print(row)

(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [ ]:
# Show 10 tuples representing which artists is the owner of which albums\
cursor.execute("SELECT Title, Name FROM albums A, artists T WHERE A.artistid == T.artistid LIMIT 10")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+---------------------------------------+----------------------+
| Title                                 | Name                 |
+---------------------------------------+----------------------+
| For Those About To Rock We Salute You | AC/DC                |
| Balls to the Wall                     | Accept               |
| Restless and Wild                     | Accept               |
| Let There Be Rock                     | AC/DC                |
| Big Ones                              | Aerosmith            |
| Jagged Little Pill                    | Alanis Morissette    |
| Facelift                              | Alice In Chains      |
| Warner 25 Anos                        | Antônio Carlos Jobim |
| Plays Metallica By Four Cellos        | Apocalyptica         |
| Audioslave                            | Audioslave           |
+---------------------------------------+----------------------+


In [ ]:
# Alternatively, join can be used instead
cursor.execute("SELECT Title, Name FROM albums A INNER JOIN artists T ON A.artistid == T.artistid LIMIT 10")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+---------------------------------------+----------------------+
| Title                                 | Name                 |
+---------------------------------------+----------------------+
| For Those About To Rock We Salute You | AC/DC                |
| Balls to the Wall                     | Accept               |
| Restless and Wild                     | Accept               |
| Let There Be Rock                     | AC/DC                |
| Big Ones                              | Aerosmith            |
| Jagged Little Pill                    | Alanis Morissette    |
| Facelift                              | Alice In Chains      |
| Warner 25 Anos                        | Antônio Carlos Jobim |
| Plays Metallica By Four Cellos        | Apocalyptica         |
| Audioslave                            | Audioslave           |
+---------------------------------------+----------------------+


## &#53;. SQL with aggregation functions

Aggregation is the function to summarize a collection of data

Five aggregate functions can be used
* count(), sum(), avg(), max(), min()

In [ ]:
# Show the number of customers records
cursor.execute("SELECT count(*) AS 'Total customer' FROM customers")
print(cursor.fetchone())

(59,)


In [ ]:
# Show the total of quantity purchases
cursor.execute("SELECT sum(Quantity) AS 'Total quantities' FROM invoice_items")
print(cursor.fetchone())

(2240,)


In [ ]:
# Show the average number of milliseconds in tracks table
cursor.execute("SELECT avg(milliseconds) AS 'Average tracks' FROM tracks")
print(cursor.fetchone())

(393599.2121039109,)


In [ ]:
# Show the maximum millisecond in tracks table
cursor.execute("SELECT max(milliseconds) FROM tracks")
print(cursor.fetchone())

(5286953,)


In [ ]:
# Show the minimum millisecond in tracks table
cursor.execute("SELECT min(milliseconds) AS 'Average tracks' FROM tracks")
print(cursor.fetchone())

(1071,)


## &#54;. SQL with nested queries

In the FROM relation, a user can embedded another SQL statement called nested query. It can be used to select tuple having attribute_value = maximum or minimum

In [ ]:
# Show the minimum millisecond in tracks table
cursor.execute("SELECT * FROM tracks, (SELECT min(milliseconds) AS mintrack FROM tracks) M WHERE milliseconds == M.mintrack")
print(cursor.fetchone())

(2461, 'É Uma Partida De Futebol', 200, 1, 1, 'Samuel Rosa', 1071, 38747, 0.99, 1071)


In [ ]:
# Show the maximum millisecond in tracks table
cursor.execute("SELECT * FROM tracks, (SELECT max(milliseconds) AS maxtrack FROM tracks) M WHERE milliseconds == M.maxtrack")
print(cursor.fetchone())

(2820, 'Occupation / Precipice', 227, 3, 19, None, 5286953, 1054423946, 1.99, 5286953)


## &#55;. SQL with derived fields

The new field can be derived from existing fields.

In [ ]:
# Compute the payment
cursor.execute("SELECT DISTINCT unitprice*quantity AS 'payment' FROM invoice_items")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+---------+
| payment |
+---------+
| 0.99    |
| 1.99    |
+---------+


In [ ]:
# Aggregate them
cursor.execute("SELECT sum(unitprice*quantity) AS 'Total payment' FROM invoice_items")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+-------------------+
| Total payment     |
+-------------------+
| 2328.599999999957 |
+-------------------+


## &#56;. SQL with group by

The new field can be derived from existing fields.

In [ ]:
# Show all customers who order items more than 40 items
cursor.execute("SELECT customerID, count(*), sum(Total) AS T FROM invoices GROUP BY CustomerID HAVING T > 40")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+------------+----------+--------------------+
| CustomerId | count(*) | T                  |
+------------+----------+--------------------+
| 5          | 7        | 40.620000000000005 |
| 6          | 7        | 49.620000000000005 |
| 7          | 7        | 42.62              |
| 24         | 7        | 43.62              |
| 25         | 7        | 42.62              |
| 26         | 7        | 47.620000000000005 |
| 28         | 7        | 43.620000000000005 |
| 37         | 7        | 43.62              |
| 43         | 7        | 40.620000000000005 |
| 44         | 7        | 41.620000000000005 |
| 45         | 7        | 45.62              |
| 46         | 7        | 45.62              |
| 48         | 7        | 40.62              |
| 57         | 7        | 46.62              |
+------------+----------+--------------------+


In [ ]:
# Show all tuples who average purchases bigger than 6
cursor.execute("SELECT * FROM (SELECT customerID, count(*), avg(Total) AS avg FROM invoices GROUP BY CustomerID) A WHERE A.avg > 6")
table = PrettyTable([description[0] for description in cursor.description])
table.align = "l"  # Set alignment for all columns (left, center, right)
for row in cursor.fetchall():
  table.add_row(row)
print(table)

+------------+----------+--------------------+
| customerID | count(*) | avg                |
+------------+----------+--------------------+
| 6          | 7        | 7.088571428571429  |
| 7          | 7        | 6.088571428571428  |
| 24         | 7        | 6.231428571428571  |
| 25         | 7        | 6.088571428571428  |
| 26         | 7        | 6.802857142857143  |
| 28         | 7        | 6.231428571428572  |
| 37         | 7        | 6.231428571428571  |
| 45         | 7        | 6.517142857142857  |
| 46         | 7        | 6.517142857142857  |
| 57         | 7        | 6.659999999999999  |
| 59         | 6        | 6.1066666666666665 |
+------------+----------+--------------------+


## Your solution

1. Show all customers
2. Show customers who make at least one purchase
3. Show address of all customers
4. Show customers who purchase above 40  
5. Show all artists  
6. Show albums that is produced by “Aerosmith”   
7. Show artists who have the total sale > 100

In [ ]:
dbconnect.close()